#The vectorial schemes for hyperbolic problems

$$
\newcommand{\R}{{\mathbb R}}
\newcommand{\DdQq}[2]{{\mathrm D}_{#1}{\mathrm Q}_{#2}}
\newcommand{\drondt}{\partial_t}
\newcommand{\drondx}{\partial_x}
\newcommand{\drondy}{\partial_y}
\newcommand{\drondtt}{\partial_{tt}}
\newcommand{\drondxx}{\partial_{xx}}
\newcommand{\dx}{\Delta x}
\newcommand{\dt}{\Delta t}
\newcommand{\grandO}{{\mathcal O}}
\newcommand{\density}[2]{\,f_{#1}^{#2}}
\newcommand{\fk}[1]{\density{#1}{\vphantom{\star}}}
\newcommand{\fks}[1]{\density{#1}{\star}}
\newcommand{\moment}[2]{\,m_{#1}^{#2}}
\newcommand{\mk}[1]{\moment{#1}{\vphantom{\star}}}
\newcommand{\mke}[1]{\moment{#1}{e}}
\newcommand{\mks}[1]{\moment{#1}{\star}}
$$

Consider the conservative hyperbolic problem
$$\drondt u(t,x) + \drondx {\cdot} f(u(t,x)) = 0, \qquad t>0, \quad x\in\R^d,$$
where $u(t,x)\in\R^N$.

A vectorial scheme can be build by coupling $N$ elementary Boltzmann schemes, one for each scalar equation. The coupling takes place in the relaxation phase and in particular in the equilibrium values of the non conserved moments.

In this work session, we investigate some classical hyperbolic systems like the shallow water and the Euler systems. 

##Shallow water in 1D

The system reads
$$\begin{aligned}&\drondt h + \drondx q = 0, \\ &\drondt q + \drondx(q^2/h+gh^2/2) = 0, \end {aligned}$$
where $g$ can be taken to $1$.
The simulation will be done on $(0,1)$ and Neumann boundary conditions will be added.

####Question 1

Propose a dictionary for a vectorial scheme build with two coupled $\DdQq{1}{2}$ for the simulation of this problem. The velocity of the scheme could be taken to $2$.

In [ ]:
import sympy as sp
import pyLBM

h, q, X, LA, g = sp.symbols('h, q, X, LA, g')
la = 2.              # velocity of the scheme
s_h, s_q = 1.7, 1.7  # relaxation parameter

dico = {

}
scheme = pyLBM.Scheme(dico)
print scheme

####Question 2

Propose a function to initialize with the Riemann problem:
$$h(0,x) = \left\{ \begin{aligned} h_L &&\text{if } 0\leq x<1/2,\\ h_R &&\text{if } 1/2<x\leq 1,  \end{aligned}\right. \qquad q(0,x) = \left\{ \begin{aligned} q_L &&\text{if } 0\leq x<1/2,\\ q_R &&\text{if } 1/2<x\leq 1,  \end{aligned}\right.$$
with $h_L=1$, $h_R=1/4$, $q_L=q_R=1/10$.

####Question 3

Compute the solution of the shallow water system with the previous scheme at $t=0.25$.

In [ ]:
import pylab as plt
%matplotlib inline

dx = 1./512          # spatial step
s_h, s_q = 1.7, 1.5  # relaxation parameter
Tf = 0.25            # final time

dico = {

}

sol = pyLBM.Simulation(dico)
x = sol.domain.x[0][1:-1]
plt.figure()
plt.clf()
plt.subplot(1,2,1)
plt.plot(x, sol.m[0][0][1:-1], 'k')
plt.title(r'$h$')
plt.subplot(1,2,2)
plt.plot(x, sol.m[1][0][1:-1], 'k')
plt.title(r'$q$')
while (sol.t<Tf):
    sol.one_time_step()
plt.subplot(1,2,1)
plt.plot(x, sol.m[0][0][1:-1], 'r')
plt.subplot(1,2,2)
plt.plot(x, sol.m[1][0][1:-1], 'b')
plt.show()

##Euler in 1D

The Euler system reads
$$\begin{aligned}&\drondt \rho + \drondx q = 0,\\ &\drondt q + \drondx \Bigl[ (\gamma-1)E + \frac{3-\gamma}{2} \frac{q^2}{\rho} \Bigr] = 0,\\ &\drondt E + \drondx \Bigl[ \gamma\frac{Eq}{\rho} - \frac{\gamma-1}{2} \frac{q^3}{\rho^2}\Bigr] = 0, \end{aligned}$$
where $\gamma=1.4$ for instance.

####Question 4

Compute the solution of this system by using a vectorial scheme composed by $3$ coupled $\DdQq{1}{2}$. The initial condition could be a Riemann problem to simulate the shock tube of Sod corresponding to
$$\rho_L = 1, \rho_R=\frac{1}{8}, p_L=1, p_R=\frac{1}{10}, u_L=u_R = 0,$$
with
$$q = \rho u, \quad E = \rho u^2 + \frac{p}{\gamma-1}.$$

In [ ]:
import sympy as sp
import pyLBM
#import pylab as plt
%matplotlib inline

def Riemann_pb(x, u_L, u_R):
    pass

# parameters
rho, q, E, X, LA = sp.symbols('rho,q,E,X,LA')
# parameters
gamma = 1.4
xmin, xmax = 0., 1.
dx = 1.e-3 # spatial step
la = 3. # velocity of the scheme
rho_L, rho_R, p_L, p_R, u_L, u_R = 1., 1./8., 1., 0.1, 0., 0.
q_L = rho_L*u_L
q_R = rho_R*u_R
E_L = rho_L*u_L**2 + p_L/(gamma-1.)
E_R = rho_R*u_R**2 + p_R/(gamma-1.)
Tf = 0.14 # final time
s_rho, s_q, s_E = 1.9, 1.5, 1.4

dico = {

}

sol = pyLBM.Simulation(dico)
while (sol.t<Tf):
    sol.one_time_step()
sol.f2m()
x = sol.domain.x[0][1:-1]
rho = sol.m[0][0][1:-1]
q = sol.m[1][0][1:-1]
E = sol.m[2][0][1:-1]
u = q/rho
p = (gamma-1.)*(E - .5*rho*u**2)
e = E/rho - .5*u**2
viewer = pyLBM.viewer.matplotlibViewer
fig = viewer.Fig(2,3)
lrho = fig[0,0].plot(x, rho, width=2, color='k')[0]
fig[0,0].axis(xmin, xmax, 0., 1.2)
fig[0,0].title = 'mass'
lu = fig[0,1].plot(x, u, width=2, color='k')[0]
fig[0,1].axis(xmin, xmax, 0., 1.)
fig[0,1].title = 'velocity'
lp = fig[0,2].plot(x, p, width=2, color='k')[0]
fig[0,2].axis(xmin, xmax, 0., 1.2)
fig[0,2].title = 'pressure'
lE = fig[1,0].plot(x, E, width=2, color='k')[0]
fig[1,0].axis(xmin, xmax, 0., 3.)
fig[1,0].title = 'energy'
lq = fig[1,1].plot(x, q, width=2, color='k')[0]
fig[1,1].axis(xmin, xmax, 0., .5)
fig[1,1].title = 'momentum'
le = fig[1,2].plot(x, e, width=2, color='k')[0]
fig[1,2].axis(xmin, xmax, 1., 3.)
fig[1,2].title = 'internal energy'
fig.show()

##Shallow water  in 2D

The system reads
$$\begin{aligned}&\drondt h + \drondx q_x + \drondy q_y = 0, \\ &\drondt q_x + \drondx(q_x^2/h+gh^2/2) + \drondy (q_xq_h/h) = 0, \\ &\drondt q_y + \drondx (q_xq_h/h) + \drondy(q_y^2/h+gh^2/2) = 0, \end {aligned}$$
where $g$ can be taken to $1$.
The simulation will be done on $(-1,1)\times(-1,1)$ and periodical boundary conditions will be added.

####Question 5

Propose a vectorial scheme build with $3$ elementary $\DdQq{2}{4}$ schemes, one for each scalar equation. The velocity of the scheme could be taken to $4$.

The initialization reads
$$h(x, y) = 1 + {\mathbf 1}_{x^2+y^2<r^2},\qquad q_x=q_y = 0,$$
where $r=1/4$.

####Question 6

Compute the solution of the shallow water with this scheme.

In [ ]:
import numpy as np
import sympy as sp
import pyLBM

X, Y, LA = sp.symbols('X, Y, LA')
h, qx, qy = sp.symbols('h, qx, qy')

def h0(x, y):
    pass

# parameters
dx = 1./128  # spatial step
la = 4.      # velocity of the scheme
h_l = 1.     # low value of the water height
h_h = 2.     # high value of the water height
L = 2        # size of the domain
g = 1.       # gravity
s_h1 = 2.
s_h2 = 1.5
s_q1 = 1.5
s_q2 = 1.2
# initialization
xmin, xmax, ymin, ymax = -.5*L, .5*L, -.5*L, .5*L
s_h = [0., s_h1, s_h1, s_h2]
s_q = [0., s_q1, s_q1, s_q2]

vitesse = range(1,5)
polynomes = [1, LA*X, LA*Y, X**2-Y**2]

dico = {

}

sol = pyLBM.Simulation(dico)
while (sol.t<.5):
    sol.one_time_step()
sol.f2m()
viewer = pyLBM.viewer.matplotlibViewer
fig = viewer.Fig()
ax = fig[0]
im = ax.image(sol.m[0][0][1:-1,1:-1].transpose())
ax.title = 'water height at t = {0:f}'.format(sol.t)
fig.show()

In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./custom.css", "r").read()
    return HTML(styles)
css_styling()